In [89]:
!pip install ndjson
!pip install cairocffi

     |████████████████████████████████| 71kB 5.0MB/s 
  Created wheel for cairocffi: filename=cairocffi-1.2.0-cp37-none-any.whl size=89548 sha256=58ff8a7c7391a13169fec66374397e71fc6945e70ee66c47e373330b60b72c59
  Stored in directory: /root/.cache/pip/wheels/40/76/48/f1effadceea83b32e7d957dd0f92db4db8b537d7b72b4ef374
Successfully built cairocffi


In [153]:
import os
import ndjson
import pandas as pd
import cairocffi as cairo
import numpy as np
import torch.utils.data as data
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from torch.utils.data import DataLoader

In [117]:
category = ['apple','face'] # add training classes
image_size = 28
number_per_catogory = 1000
dataset_seed = 42



In [136]:
# credit to:
#https://github.com/googlecreativelab/quickdraw-dataset/issues/19

def vector_to_raster(vector_images, side=28, line_diameter=16, padding=16, bg_color=(0,0,0), fg_color=(1,1,1)):
    """
    padding and line_diameter are relative to the original 256x256 image.
    """
    
    original_side = 256.
    
    surface = cairo.ImageSurface(cairo.FORMAT_ARGB32, side, side)
    ctx = cairo.Context(surface)
    ctx.set_antialias(cairo.ANTIALIAS_BEST)
    ctx.set_line_cap(cairo.LINE_CAP_ROUND)
    ctx.set_line_join(cairo.LINE_JOIN_ROUND)
    ctx.set_line_width(line_diameter)

    # scale to match the new size
    # add padding at the edges for the line_diameter
    # and add additional padding to account for antialiasing
    total_padding = padding * 2. + line_diameter
    new_scale = float(side) / float(original_side + total_padding)
    ctx.scale(new_scale, new_scale)
    ctx.translate(total_padding / 2., total_padding / 2.)

    raster_images = []
    for vector_image in vector_images:
        # clear background
        ctx.set_source_rgb(*bg_color)
        ctx.paint()
        
        bbox = np.hstack(vector_image).max(axis=1)
        offset = ((original_side, original_side) - bbox) / 2.
        offset = offset.reshape(-1,1)
        centered = [stroke + offset for stroke in vector_image]

        # draw strokes, this is the most cpu-intensive part
        ctx.set_source_rgb(*fg_color)        
        for xv, yv in centered:
            ctx.move_to(xv[0], yv[0])
            for x, y in zip(xv, yv):
                ctx.line_to(x, y)
            ctx.stroke()

        data = surface.get_data()
        raster_image = np.copy(np.asarray(data)[::4])
        raster_images.append(raster_image)
    
    return raster_images

In [20]:
# download simplified data
!mkdir quickDrawData
for item in category:
  path=os.path.join('gs://quickdraw_dataset/full/simplified',item+'.ndjson')
  print(path)
  !gsutil -m cp $path ./quickDrawData/

gs://quickdraw_dataset/full/simplified/apple.ndjson
Copying gs://quickdraw_dataset/full/simplified/apple.ndjson...
/ [1/1 files][ 56.1 MiB/ 56.1 MiB] 100% Done                                    
Operation completed over 1 objects/56.1 MiB.                                     
gs://quickdraw_dataset/full/simplified/airplain.ndjson
CommandException: No URLs matched: gs://quickdraw_dataset/full/simplified/airplain.ndjson
CommandException: 1 file/object could not be transferred.
gs://quickdraw_dataset/full/simplified/face.ndjson
Copying gs://quickdraw_dataset/full/simplified/face.ndjson...
\ [1/1 files][ 89.4 MiB/ 89.4 MiB] 100% Done                                    
Operation completed over 1 objects/89.4 MiB.                                     


In [157]:
class DrawDataset(data.Dataset):
    def __init__(self, image_size, categories, normalize, transformation, 
                 number_per_catogory):
        all_images = []
        all_labels = []
        all_keys = []
        for item in categories:
          with open(os.path.join('quickDrawData',item+'.ndjson')) as f:
            print(item)
            data = ndjson.load(f)
            df=pd.DataFrame.from_dict(data)
            new_df=df[df['countrycode']=='CA']
            new_df=new_df[new_df['recognized']==True]
            all_images+=(list(new_df['drawing'].values)[:number_per_catogory])
            all_labels+=(list(new_df['word'].values)[:number_per_catogory])
            all_keys+=(list(new_df['key_id'].values)[:number_per_catogory])

        arr=vector_to_raster(all_images,side=image_size)
        images = [x.reshape(image_size,image_size) for x in arr]
        if transformation==False:
          if normalize==True:
            images=images/255

        label_encoder = LabelEncoder()
        integer_encoded = label_encoder.fit_transform(all_labels)
        onehot_encoder = OneHotEncoder(sparse=False)
        integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
        onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
        
        self.X = images
        self.y = onehot_encoded
        self.labels = all_labels
        self.keys = all_keys
        # normalize

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return self.X[index],self.y[index]

In [158]:
dataset=DrawDataset(image_size, category, False, False, 
                 number_per_catogory)

apple
face


In [166]:
def load_data(dataset, batch_size, dataset_seed):

    train_set, val_set = data.random_split(dataset, 
                        [int(np.floor(len(dataset)*0.7)), int(np.floor(len(dataset)*0.3))],
                        generator=torch.Generator().manual_seed(dataset_seed))

    train_loader=DataLoader(train_set, batch_size=batch_size,shuffle=True)
    val_loader=DataLoader(val_set, batch_size=batch_size,shuffle=False)
    return train_loader, val_loader


In [167]:
dataset=DrawDataset(image_size, category, False, False, 
                 number_per_catogory)

apple
face


In [170]:
batch_size = 32
train_loader, val_loader= load_data(dataset,batch_size,dataset_seed)

In [146]:
# look at an image
from PIL import Image
img = Image.fromarray(images[1100])
img.save('1100.png')
